In [1]:
# Importing the required libraries

#import community as community_louvain

import numpy as np
import networkx as nx
import scipy as sp

import networkx.algorithms.community as nx_comm
%matplotlib inline


In [38]:
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [218]:
NODES_FILE = './nodes_repo_community.csv'
NODES_FILE1 = './nodes_repo_Com_FFForks.csv'

In [4]:
G = nx.read_edgelist('./edges.csv',
                     create_using=nx.DiGraph(), delimiter = ', ')



In [219]:
#add node attribute community with corresponding community label #
nodesFile = open(NODES_FILE, 'r')
record = {}
# skip headers
#nodesFile.readline()

# store nodes in record
for line in nodesFile:
    toRecord = line.split(", ")
    #print(toRecord)
    if (toRecord[2] != '-1\n'):
        record[toRecord[0]] = int(toRecord[2].strip("\n"))
nx.set_node_attributes(G,record,"communities")   

#print(record)
#print(G)
nodesFile.close()

In [7]:
#create adj matrix
SP_adj_matrix = nx.adjacency_matrix(G)
NP_adj_matrix = SP_adj_matrix.todense()
print(NP_adj_matrix)
print(type(NP_adj_matrix))
print(NP_adj_matrix.shape)

[[0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'numpy.matrix'>
(3010, 3010)


In [59]:
#create list that is the vector for community affiliation
community_affiliation = []
for i in range(len(NP_adj_matrix)):
    community_affiliation.append(list(G.nodes("communities"))[i][1])

3010


FUNCTION BELOW IS MODIFIED BUT TAKEN FROM: https://github.com/CarloNicolini/pyconnectivity/blob/master/bct.py

In [123]:
def participation_coef(W, ci):
    '''
    Participation coefficient is a measure of diversity of intermodular
    connections of individual nodes.
    Inputs:     W,      binary/weighted, directed/undirected connection matrix
               Ci,         community affiliation vector
    Output:     P,      participation coefficient
    Note: The output for directed graphs is the "out-neighbor"
         participation coefficient.
    '''
  

    _, ci = np.unique(ci, return_inverse=True)
    ci += 1
   
    n = len(W)  # number of vertices
    Ko = np.sum(W, axis=1)  # (out) degree
    
    Gc = np.dot((W != 0), np.diag(ci))  # neighbor community affiliation
    
    Kc2 = np.zeros((n,1))  # community-specific neighbors
   
    for i in range(int(np.max(ci))):
        Kc2 += np.sqrt(np.sum(W * (Gc == i), axis=1)) #add more sqrt if necessary (if values still negative)
        checker,_ = np.where(Kc2<0) #check to see if any values of Kc overflow and become negative
        if(len(checker)>0):
            print("OVERFLOW. TRY AGAIN WITH MORE SQRT")
            break
        if(i % 5 == 0 or i==(int(np.max(ci)))-1):
            print(str(i) + " out of " + str(np.max(ci)-1) + " communities complete")
        
    non_normal = Kc2 / np.sqrt(Ko) 
    non_normal = np.nan_to_num(non_normal,nan=np.nanmax(non_normal)) #change any nan values to max value in vector
    normalized = (non_normal - non_normal.min()) / (non_normal.max() - non_normal.min()) #normalized vector
    P = np.ones((n,1)) - normalized
    
    #P=0 if for nodes with no (out) neighbors
    P[np.where(np.logical_not(Ko))] = 0

    return P

In [124]:
Pc = participation_coef(NP_adj_matrix,community_affiliation)

0 out of 143 communities complete
5 out of 143 communities complete
10 out of 143 communities complete
15 out of 143 communities complete
20 out of 143 communities complete
25 out of 143 communities complete
30 out of 143 communities complete
35 out of 143 communities complete
40 out of 143 communities complete
45 out of 143 communities complete
50 out of 143 communities complete
55 out of 143 communities complete
60 out of 143 communities complete
65 out of 143 communities complete
70 out of 143 communities complete
75 out of 143 communities complete
80 out of 143 communities complete
85 out of 143 communities complete
90 out of 143 communities complete
95 out of 143 communities complete
100 out of 143 communities complete
105 out of 143 communities complete
110 out of 143 communities complete
115 out of 143 communities complete
120 out of 143 communities complete
125 out of 143 communities complete
130 out of 143 communities complete
135 out of 143 communities complete
140 out of 143

In [125]:
print(Pc)

[[0.86303287]
 [0.76533999]
 [0.        ]
 ...
 [0.        ]
 [0.96924893]
 [0.96563587]]


In [248]:
#write PC to new file
nodesFile1 = open(NODES_FILE1, 'r')
record1 = {}


# store nodes in record
for line in nodesFile1:
    toRecord1 = line.split(", ")
    user1 = toRecord1[0].strip()
    repo1 = toRecord1[1].strip()
    follower_community1 = toRecord1[2].strip()
    fork_community1 = toRecord1[3].strip()
    follower_pc1 = -1
    
    record1[toRecord1[0]] = [repo1,follower_community1,fork_community1,follower_pc1]

nodesFile1.close()


In [249]:
# make dict with user as key and value as PC
nodes = list(G.nodes())

pc_record = {}

for i in range(len(nodes)):
    
    pc_record[nodes[i]] = Pc.item(i)


In [246]:
#write PC of each node into excel nodeslist
OUTPUT_NODES_FILE = './nodes_communities_PC1.csv'

counter = 0
for contributor in pc_record:

    record1[contributor][3] = Pc.item(counter)
    
    counter +=1

newNodesFile = open(OUTPUT_NODES_FILE, 'w+')

for user in record1:
    newNodesFile.write(f"{user}, {record1[user][0]}, {record1[user][1]}, {record1[user][2]}, {record1[user][3]}\n")
newNodesFile.close()

In [126]:
#find avg participation coefficient of the whole network
avg_pc = np.mean(Pc)
print(avg_pc)

0.663927310367459


In [189]:
#find max and min average community participation coefficient
#list size is amount of communities
community_pc = [0] * int(np.max(community_affiliation)+1)
#print(Pc)
#print(Pc.item(0))
#print(len(community_pc))
#print(community_affiliation)
#number of nodes in each community to calc average
num_of_nodes_in_comm = [0] * int(np.max(community_affiliation)+1) #size of 144
#print(len(NP_adj_matrix))
#For each community find the average PC
for i in range(int(np.max(community_affiliation))+1): 
    #for each node get their PC 
    for j in range(len(NP_adj_matrix)):
        #if node is in community i 
        if(community_affiliation[j] == i):
            #then add PC to community_pc
            community_pc[i] += Pc.item(j)
            #add 1 to index 
            num_of_nodes_in_comm[community_affiliation[j]] += 1      
print(community_pc)
print(num_of_nodes_in_comm)            
avg_community_pc = [0] * int(np.max(community_affiliation)+1) #144

#average out each element of avg_community_pc
for i in range(len(avg_community_pc)):
    #PC[i] / # of nodes in community igy
    avg_community_pc[i] = community_pc[i]/num_of_nodes_in_comm[i]
    

#print(avg_community_pc)
print(np.max(avg_community_pc))
print(np.min(avg_community_pc))
print(np.where(avg_community_pc ==np.max(avg_community_pc) ))
print(np.where(avg_community_pc ==np.min(avg_community_pc) ))
    

[246.38574499090467, 9.85519475238171, 1.8731266364113999, 3.9272409018429646, 1.659408745083423, 75.02914502906783, 1.8351960935158487, 1.029175288008681, 2.4178780485850826, 2.5977539853458653, 2.762433416152514, 0.8439389008668369, 6.688458227825932, 1.9073334133701012, 1.8731266364113999, 271.79443455217273, 1.9657931969009397, 1.0, 1.0, 216.99501131754653, 1.0, 299.59382293049214, 3.733270858072203, 2.717476339051221, 1.0, 3.7864615061969955, 126.83974088380528, 1.632470339888954, 1.0, 0.8907810492010726, 1.802653180898159, 1.8395495024820512, 1.0, 1.0, 1.8422417704975316, 4.565237828448277, 1.9657931969009397, 1.8348891366592688, 1.7650056136287267, 2.5970057920354206, 5.95641946429374, 5.844884696835315, 2.758324243847167, 2.0, 1.9315863938018794, 1.8952905583534856, 1.8435067689180262, 1.0, 1.8112300553600709, 2.6099200819642094, 2.6215991488668515, 1.9315863938018794, 1.9036174859314325, 286.26029506391774, 0.9173286984480348, 2.7785099217882716, 1.0, 1.9073334133701012, 5.168